## Transform Data using Spark APIs

Let us transform the data using Spark APIs to add new columns using timestamp from the log messages so that data can be partitioned by year, month and then by day of month while writing to target location.
* We will start with creating spark session.
* Using the spark session's `readStream` we will subscribe to Kafka Topic to create streaming Data Frame.
* We can apply required transformations to add columns such as year, month and dayofmonth using the timestamp that is part of each and every message.
* As part of the next lecture we will see how to partition the data using these new columns while writing to target. For now, we will validate whether new columns are added or not using `memory` as the target along with `queryName`.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1'). \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Kafka and Spark Integration'). \
    master('yarn'). \
    getOrCreate()

In [2]:
kafka_bootstrap_servers = 'w01.itversity.com:9092,w02.itversity.com:9092'

In [3]:
df = spark. \
  readStream. \
  format('kafka'). \
  option('kafka.bootstrap.servers', kafka_bootstrap_servers). \
  option('subscribe', f'{username}_retail'). \
  load()

In [4]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
from pyspark.sql.functions import lit, date_format, to_date, split, substring

In [6]:
l = [('X',)]

In [7]:
dual = spark.createDataFrame(l, schema='dummy STRING')
dual.show(10, False)

+-----+
|dummy|
+-----+
|X    |
+-----+



In [8]:
dual.select(to_date(lit('2021-Jan-21'), 'yyyy-MMM-dd')).show()

+-------------------------------------+
|to_date('2021-Jan-21', 'yyyy-MMM-dd')|
+-------------------------------------+
|                           2021-01-21|
+-------------------------------------+



In [9]:
dual.select(to_date(lit('31/Dec/2021:00:37:39'), 'dd/MMM/yyyy:HH:mm:ss')).show()

+-------------------------------------------------------+
|to_date('31/Dec/2021:00:37:39', 'dd/MMM/yyyy:HH:mm:ss')|
+-------------------------------------------------------+
|                                             2021-12-31|
+-------------------------------------------------------+



In [10]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    writeStream. \
    format("memory"). \
    queryName("log_messages_raw"). \
    start()

In [13]:
spark.sql('SELECT * FROM log_messages_raw').show(truncate=False)

+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                 |
+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|null|58.138.168.125 - - [23/Feb/2023:11:45:38 -0800] "GET /categories/tennis%20%26%20racquet/products HTTP/1.1" 200 338 "-" "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"|
|null|148.171.173.137 - - [23/Feb/2023:11:45:39 

In [26]:
spark.sql('SELECT count(1) FROM log_messages_raw').show(truncate=False)

+--------+
|count(1)|
+--------+
|20000   |
+--------+



In [ ]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"). \
    withColumn('log_date', to_date(substring(split('value', ' ')[3], 2, 21), 'dd/MMM/yyyy:HH:mm:ss')). \
    withColumn('year', date_format('log_date', 'yyyy')). \
    withColumn('month', date_format('log_date', 'MM')). \
    withColumn('dayofmonth', date_format('log_date', 'dd')). \
    writeStream. \
    format("memory"). \
    queryName("log_messages"). \
    start()

In [24]:
spark.sql('SELECT * FROM log_messages').show(truncate=False)

+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----+-----+----------+
|key |value                                                                                                                                                                                                                                  |log_date  |year|month|dayofmonth|
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----+-----+----------+
|null|208.81.232.6 - - [23/Feb/2023:06:43:39 -0800] "GET /departments HTTP/1.1" 200 1004 "-" "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.191

In [27]:
spark.sql('SELECT count(1) FROM log_messages').show(truncate=False)

+--------+
|count(1)|
+--------+
|20000   |
+--------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56962)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/accumulato